In [ ]:
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd
import re

In [ ]:
# Title of job in search
title = "Data Scientist"

# Location
location = "Los Angeles County"

In [ ]:
# link for list of job postings on the left on linkedin
list_url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=data%2Bscientist&location=los%2Bangeles%2Bcounty&geoId=&trk=public_jobs_jobs-search-bar_search-submit&start=25"

response = requests.get(list_url)

list_data = response.text
list_soup = BeautifulSoup(list_data, "html.parser")
page_jobs = list_soup.findAll("li")

In [ ]:
job_id_list = []

for job in page_jobs:
    # find job_ids
    base_card_div = job.find("div", {"class": "base-card"})
    job_id = base_card_div.get("data-entity-urn").split(":")[3]
    job_id_list.append(job_id)
    

In [ ]:
job_list = []
for job_id in job_id_list:
    job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
    job_response = requests.get(job_url)
    job_soup = BeautifulSoup(job_response.text, "html.parser")

    # Dictionary for each job posting
    job_post = {}
    
    # company name
    try:
        job_post["company_name"] = job_soup.find("a",{"class": "topcard__org-name-link topcard__flavor--black-link"}).text.strip()
    except:
        job_post["company_name"] = None
    
    # company location
    try: 
        job_post["location"] = job_soup.find("span", {"class": "topcard__flavor topcard__flavor--bullet"}).text.strip()
    except:
        job_post["location"] = None

    # Salary
    try:
        job_post["salary"] = job_soup.find("div", {"class": "salary compensation__salary"}).text.strip()
    
    except AttributeError:
        description = job_soup.find("div", {"class": "show-more-less-html__markup show-more-less-html__markup--clamp-after-5 relative overflow-hidden"}).text
        job_post["salary"] = description
        pattern = r"\$[\d,]+(?:\.\d{2})?\s?to\s?\$[\d,]+(?:\.\d{2})?"
        salary_range = re.findall(pattern, description)
        job_post["salary"] = salary_range

    # Description
    

    job_list.append(job_post)

In [ ]:
job_list

In [ ]:
test_url = "https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/4182723167"
test_response = requests.get(test_url)
test_soup = BeautifulSoup(test_response.text, "html.parser")

description = test_soup.find("div", {"class": "show-more-less-html__markup show-more-less-html__markup--clamp-after-5 relative overflow-hidden"})

In [ ]:
description.text